In [3]:
# Instalar bitcoinlib (ya está instalado)
# !pip install bitcoinlib

import hashlib
import struct
from bitcoinlib.wallets import Wallet
from bitcoinlib.transactions import Transaction
from bitcoinlib.keys import HDKey
# Corrección: Services se importa diferente
from bitcoinlib.services.services import Service  # ← Opción 1

# Si la opción 1 no funciona, usa esto:
# from bitcoinlib import services
# service = services.Service()

# Función para aplicar double SHA-256
def double_sha256(data):
    return hashlib.sha256(hashlib.sha256(data).digest()).digest()

# Función para calcular el Merkle Root
def merkle_root(transactions):
    if not transactions:
        return b''
    while len(transactions) > 1:
        new_transactions = []
        for i in range(0, len(transactions), 2):
            if i + 1 < len(transactions):
                combined = transactions[i] + transactions[i + 1]
            else:
                combined = transactions[i] + transactions[i]
            new_transactions.append(double_sha256(combined))
        transactions = new_transactions
    return transactions[0]

# Crear/abrir billetera
try:
    wallet = Wallet.create('test_wallet_v2', network='bitcoin')
    print("✅ Billetera creada")
except Exception as e:
    try:
        wallet = Wallet('test_wallet_v2')
        print("✅ Billetera abierta")
    except:
        print(f"❌ Error: {e}")
        wallet = None

if wallet:
    new_key = wallet.new_key()
    address = new_key.address
    print(f"📍 Dirección: {address}")

# ============================================
# Ejemplo educativo: Calcular Merkle Root
# ============================================

transactions_hex = [
    '4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b',
]

transactions_bin = [bytes.fromhex(tx) for tx in transactions_hex]
new_merkle_root = merkle_root(transactions_bin)

print(f"\n📊 Merkle Root: {new_merkle_root.hex()}")

# Header del bloque
def create_block_header(version, prev_hash, merkle_root, timestamp, bits, nonce):
    return (struct.pack('<I', version) +
            prev_hash +
            merkle_root +
            struct.pack('<I', timestamp) +
            struct.pack('<I', bits) +
            struct.pack('<I', nonce))

version = 1
prev_hash = bytes(32)
timestamp = 1231006505
bits = 0x1d00ffff
nonce = 2083236893

new_header = create_block_header(version, prev_hash, new_merkle_root, timestamp, bits, nonce)

print(f"📦 Block Header: {new_header.hex()}")

block_hash = double_sha256(new_header)
print(f"🔗 Block Hash: {block_hash[::-1].hex()}")

✅ Billetera creada
📍 Dirección: bc1qaq5jxmp646h5f9thxlm26xenx4fvuee5tfwqr5

📊 Merkle Root: 4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b
📦 Block Header: 0100000000000000000000000000000000000000000000000000000000000000000000004a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b29ab5f49ffff001d1dac2b7c
🔗 Block Hash: f3554f2f2af964264669e106f2367c27fe48e49b767f48ca6f0166d0393dc6f2
